# **Notebook : Preparation for Differential Expression - TCGA-UCEC**

Ce notebook prépare les données TCGA-UCEC pour l'analyse différentielle avec DESeq2.

Il organise les matrices de comptage et les métadonnées cliniques pour l'export vers R.

---

**Pipeline Overview:**
- Setup and Configuration
- Load Processed Data
- Data Organization
- Quality Checks
- Export for R/DESeq2 Analysis
- Summary

<Small>## Organisation des données pour l'analyse différentielle

### Objectif scientifique
Préparer les données d'expression génique TCGA-UCEC et les métadonnées cliniques
pour l'analyse différentielle (tumeur vs normal) avec DESeq2 dans R.

### Logique méthodologique
Cette étape organise :
- Les matrices de comptage RNA-seq filtrées et normalisées
- Les métadonnées cliniques (type d'échantillon, variables cliniques)
- Les annotations des gènes

Aucune normalisation DESeq2 n'est appliquée ici (sera faite dans R).
Les comptages bruts sont conservés pour DESeq2.

### Résultat attendu
- Une matrice de comptage (gènes × échantillons) au format CSV
- Une table de métadonnées cliniques
- Des objets prêts pour l'analyse différentielle DESeq2
</Small>


# **Setup block**


### Setup and Configuration

In [1]:
# ==========================================================================================================
# GESTION SYSTÈME & ENVIRONNEMENT
import os                                       # Navigation fichiers (DIRS, chemins relatifs)
import gc                                       # Gestion mémoire (nettoyage objets inutilisés)
import json                                     # Lecture du dictionnaire de métadonnées (pipeline / EDA)  
import warnings                                 # Masquer warnings Scanpy/AnnData (dépréciation)
import re                                       # Parsing noms échantillons (regex pour donor_id)
from IPython.display import Markdown, display   # Affichage Jupyter (titres formatés, tableaux HTML)

# ----------------------------------------------------------------------------------------------------------
# CALCUL NUMÉRIQUE & VISUALISATION
# >> Générer les QC plots (n_genes vs n_counts, % mitochondrial, distributions par pathologie)
import numpy as np                              # Matrices creuses (X sparse), seuils QC (percentiles)
import math                                     # Calculs grilles subplots (ceil/floor pour layout)
import matplotlib.pyplot as plt                 # Figures principales (violin, scatter, hist)
from matplotlib.colors import to_rgb, rgb_to_hsv, hsv_to_rgb, rgb2hex
from matplotlib.gridspec import GridSpec
import seaborn as sns                           # Heatmaps corrélations, boxplots stylisés

# ----------------------------------------------------------------------------------------------------------
# ANALYSE SINGLE-CELL
# >> Charger données brutes (ad.read_h5ad), appliquer filtres QC (mitochondrial %, doublets),
#    calculer métriques (sc.pp.calculate_qc_metrics)
import anndata as ad                            # Objet AnnData (adata.obs, adata.var, adata.X)
import scanpy as sc                             # Fonctions QC (sc.pp.filter_cells, sc.pl.violin)
import pandas as pd                             # Métadonnées (fusion adata.obs), tableaux récapitulatifs

# ----------------------------------------------------------------------------------------------------------
# CLUSTERING (PRÉPARATION POUR S2 - NON UTILISÉ DANS S1)
# >> Importé par anticipation pour pipeline complet (normalisation → Leiden)
import leidenalg                                # Algorithme Leiden (clustering post-normalisation)
import igraph                                   # Graphe k-NN (requis pour leidenalg sous Scanpy)

# ==========================================================================================================
# --- DÉFINITION DES DOSSIERS ---
PROJECT_ROOT = "C:\\Z\\M2_AIDA\\transcriptomics_project"  # Laïla

DIRS = {
    "DATA":    os.path.join(PROJECT_ROOT, "data"),
    "EDA":     os.path.join(PROJECT_ROOT, "eda"),
    "FIGURES": os.path.join(PROJECT_ROOT, "figures"),
    "TMP":     os.path.join(PROJECT_ROOT, "tmp_cache")
}

for path in DIRS.values():
    os.makedirs(path, exist_ok=True)

os.chdir(PROJECT_ROOT)

# ==========================================================================================================
# --- PARAMÈTRES SCANPY ---
sc.settings.figdir = DIRS["FIGURES"]
sc.settings.cachedir = DIRS["TMP"]
sc.settings.datasetdir = DIRS["DATA"]
sc.settings.set_figure_params(dpi=100, frameon=False)

warnings.filterwarnings("ignore") 

# ==========================================================================================================
print(f"✅ Environnement chargé. Working directory: {os.getcwd()}")

✅ Environnement chargé. Working directory: c:\Z\M2_AIDA\transcriptomics_project


# **Data Loading**

### Dataset analytique normazlisé + UMAP exploitable pour DE :

<br>  local:       `data/HBCC_SCZ_CTRL_postQC_UMAP.h5ad`.

In [ ]:
dataset_path = os.path.join(DIRS["DATA"], "HBCC_SCZ_CTRL_postQC_UMAP.h5ad")
# adata = sc.read_h5ad(dataset_path) >>>>> MemoryError !
adata = sc.read_h5ad(dataset_path, backed='r')  # Lecture en mode backed pour économiser la mémoire
# ** Note technique :** backed mode pour éviter MemoryError.

dictionary_path = os.path.join(DIRS["DATA"],"hbcc_cellxgene_metadata_dictionary.json")
with open(dictionary_path, "r") as f:
    meta_hbcc = json.load(f)

print(f"✅ HBCC_SCZ_CTRL_postQC_UMAP and metadata dictionary successfully loaded (backed mode). Shape (cells, genes): {adata.shape}")

✅ HBCC_SCZ_CTRL_postQC_UMAP and metadata dictionary successfully loaded (backed mode). Shape (cells, genes): (209093, 34176)


## XXX

#### **Pseudobulk Aggregation by Cell Type and Donor**

<small>
Cette étape vise à résumer l’expression génique du cortex préfrontal humain en profils représentatifs par **donneur** et par **type cellulaire**, à partir des données single-cell de la cohorte HBCC. L’objectif est de disposer, pour chaque type cellulaire, d’une mesure globale d’expression comparable entre individus.  
<br><br>
L’agrégation par donneur permet de replacer l’analyse à l’échelle de l’individu, qui constitue l’unité d’intérêt pour la comparaison entre patients atteints de schizophrénie (SCZ) et sujets contrôles (CTRL). Les types cellulaires considérés correspondent aux annotations de référence fournies avec l’atlas HBCC.  
<br><br>
Cette étape produit des matrices d’expression synthétiques par type cellulaire, qui serviront de support aux analyses comparatives ultérieures entre groupes diagnostiques.
</small>


In [36]:
from scipy.sparse import csr_matrix
import pandas as pd
import numpy as np
import gc

# ============================================================================
# ÉTAPE 1-3 : CHARGER ET PRÉPARER LES MÉTADONNÉES (DÉJÀ FAIT)
# ============================================================================
# (Garder tout le code jusqu'à metadata_full avec sample_id)

print("\n🔄 Agrégation pseudobulk CELLULE PAR CELLULE (ultra-économique)...")

# Créer sample_id
metadata_full['sample_id'] = (
    metadata_full['cell_type'].astype(str) + "__" + 
    metadata_full['donor_id'].astype(str)
)

unique_samples = metadata_full['sample_id'].unique()
print(f"✅ {len(unique_samples)} échantillons pseudobulk à créer")

# Initialiser les accumulateurs
pseudobulk_dict = {}
for sample in unique_samples:
    pseudobulk_dict[sample] = np.zeros(adata.n_vars, dtype=np.float64)

# Reset index pour avoir des positions numériques
metadata_full = metadata_full.reset_index(drop=True)

# ============================================================================
# AGRÉGATION CELLULE PAR CELLULE
# ============================================================================

n_cells = len(metadata_full)
print(f"📊 Traitement de {n_cells:,} cellules...")

for i in range(n_cells):
    # Lire UNE seule cellule
    try:
        cell_counts = adata.X[i, :].toarray().flatten()
    except:
        # Si toarray() échoue, essayer autre chose
        cell_counts = np.asarray(adata.X[i, :]).flatten()
    
    # Récupérer le sample_id
    sample_id = metadata_full.loc[i, 'sample_id']
    
    # Accumuler
    pseudobulk_dict[sample_id] += cell_counts
    
    # Afficher progression
    if (i + 1) % 10000 == 0:
        pct = 100 * (i + 1) / n_cells
        print(f"  Progression : {i+1:,}/{n_cells:,} cellules ({pct:.1f}%)")
        gc.collect()

print("✅ Agrégation terminée")

# Convertir en DataFrame
pseudobulk_counts = pd.DataFrame.from_dict(
    pseudobulk_dict,
    orient='index',
    columns=adata.var_names
)

print(f"📊 Matrice finale : {pseudobulk_counts.shape[0]} échantillons × {pseudobulk_counts.shape[1]} gènes")

# ============================================================================
# ÉTAPE 5-7 : MÉTADONNÉES, FILTRES, EXPORT (CODE IDENTIQUE)
# ============================================================================

# Métadonnées
pseudobulk_metadata = (
    metadata_full
    .groupby('sample_id')
    .first()[['cell_type', 'donor_id', 'disease']]
)

n_cells = metadata_full.groupby('sample_id').size()
pseudobulk_metadata['n_cells'] = n_cells

print("\n📊 Distribution par type cellulaire :")
print(pseudobulk_metadata['cell_type'].value_counts())

# FILTRE 1
MIN_CELLS = 20
mask_cells = pseudobulk_metadata['n_cells'] >= MIN_CELLS
removed = (~mask_cells).sum()

pseudobulk_metadata = pseudobulk_metadata[mask_cells]
pseudobulk_counts = pseudobulk_counts.loc[pseudobulk_metadata.index]

print(f"\n✅ FILTRE 1 (≥{MIN_CELLS} cellules) : {removed} échantillons retirés")

# FILTRE 2
MIN_DONORS = 3

donors_per_group = (
    pseudobulk_metadata
    .groupby(['cell_type', 'disease'])['donor_id']
    .nunique()
    .unstack(fill_value=0)
)

print("\n📊 Donneurs par groupe :")
print(donors_per_group)

valid_cell_types = []
for ct in pseudobulk_metadata['cell_type'].unique():
    subset = pseudobulk_metadata[pseudobulk_metadata['cell_type'] == ct]
    n_ctrl = subset[subset['disease'] == 'normal']['donor_id'].nunique()
    n_scz = subset[subset['disease'] == 'schizophrenia']['donor_id'].nunique()
    
    if n_ctrl >= MIN_DONORS and n_scz >= MIN_DONORS:
        valid_cell_types.append(ct)
        print(f"✅ {ct} : CTRL={n_ctrl}, SCZ={n_scz}")
    else:
        print(f"❌ {ct} : CTRL={n_ctrl}, SCZ={n_scz} [EXCLU]")

mask_valid = pseudobulk_metadata['cell_type'].isin(valid_cell_types)
pseudobulk_metadata = pseudobulk_metadata[mask_valid]
pseudobulk_counts = pseudobulk_counts.loc[pseudobulk_metadata.index]

print(f"\n✅ FILTRE 2 : {len(valid_cell_types)} types cellulaires retenus")

# EXPORT
print("\n💾 Export des fichiers...")

export_dir = os.path.join(DIRS["DATA"], "exports")
os.makedirs(export_dir, exist_ok=True)

counts_export = pseudobulk_counts.T
counts_path = os.path.join(export_dir, "pseudobulk_counts.csv")
counts_export.to_csv(counts_path)
print(f"✅ {counts_path}")

metadata_path = os.path.join(export_dir, "pseudobulk_metadata.csv")
pseudobulk_metadata.to_csv(metadata_path)
print(f"✅ {metadata_path}")

print("\n" + "="*70)
print("✅ AGRÉGATION PSEUDOBULK TERMINÉE")
print("="*70)
print(f"📊 {pseudobulk_counts.shape[1]} échantillons")
print(f"🧬 {pseudobulk_counts.shape[0]} gènes")
print(f"🔬 {len(valid_cell_types)} types cellulaires")
print("="*70)


🔄 Agrégation pseudobulk CELLULE PAR CELLULE (ultra-économique)...
✅ 4282 échantillons pseudobulk à créer
📊 Traitement de 280,455 cellules...
  Progression : 10,000/280,455 cellules (3.6%)
  Progression : 20,000/280,455 cellules (7.1%)
  Progression : 30,000/280,455 cellules (10.7%)
  Progression : 40,000/280,455 cellules (14.3%)
  Progression : 50,000/280,455 cellules (17.8%)
  Progression : 60,000/280,455 cellules (21.4%)
  Progression : 70,000/280,455 cellules (25.0%)
  Progression : 80,000/280,455 cellules (28.5%)
  Progression : 90,000/280,455 cellules (32.1%)
  Progression : 100,000/280,455 cellules (35.7%)
  Progression : 110,000/280,455 cellules (39.2%)
  Progression : 120,000/280,455 cellules (42.8%)
  Progression : 130,000/280,455 cellules (46.4%)
  Progression : 140,000/280,455 cellules (49.9%)
  Progression : 150,000/280,455 cellules (53.5%)
  Progression : 160,000/280,455 cellules (57.1%)
  Progression : 170,000/280,455 cellules (60.6%)
  Progression : 180,000/280,455 cell

#### **Interpretation of Pseudobulk Aggregation by Cell Type and Donor**

<small>
L'agrégation pseudobulk a été réalisée pour créer des profils d'expression globaux par **donneur** et **type cellulaire**, en sommant les comptages de noyaux similaires. Cette approche permet de préparer des données adaptées à l'analyse différentielle tout en évitant la pseudo-réplication cellulaire.

**Objectif** :  
Créer des unités biologiquement indépendantes (donneur) pour les comparaisons entre les groupes **schizophrénie (SCZ)** et **contrôles (CTRL)**.

**Méthodologie** :  
Les données ont été agrégées à l’échelle du donneur et du type cellulaire, sans transformation ni normalisation supplémentaire. Cette étape produit une matrice comparable à des données bulk RNA-seq.

**Filtres appliqués** :  
1. **Filtre 1** : Retrait des échantillons avec moins de **20 cellules**.
2. **Filtre 2** : Exclusion de types cellulaires avec trop peu de cellules dans un des groupes diagnostiques.

**Résultats** :  
- **18 types cellulaires** retenus.
- **34176 échantillons** pseudobulk, pour **2234 gènes**.
- **Export des résultats** : Fichiers CSV générés pour les comptages (`pseudobulk_counts.csv`) et les métadonnées (`pseudobulk_metadata.csv`).

Cette agrégation est prête pour les analyses différentielles entre les groupes SCZ et CTRL.
</small>


#### **Differential Expression Analysis (SCZ vs CTRL) by Cell Type**

<small>
L’objectif de cette étape est d’identifier les gènes dont l’expression diffère de manière significative entre les groupes **schizophrénie (SCZ)** et **contrôles (CTRL)**, pour chaque **type cellulaire** dans la cohorte HBCC. L'analyse est réalisée à l’échelle du **donneur**, chaque donneur étant considéré comme une unité statistique. Cette approche permet de contrôler les variations inter-individus tout en analysant les différences au niveau des types cellulaires.

La méthode utilisée repose sur un modèle linéaire, où l’on teste les différences d’expression entre SCZ et CTRL, avec une correction pour les covariables pertinentes, telles que l'âge, le sexe, etc. Les résultats sont validés en contrôlant le taux de faux positifs par une correction FDR.

Les résultats de cette analyse fourniront une liste de gènes différentiellement exprimés, avec des informations sur l'effet estimé, les valeurs p et les valeurs ajustées.
</small>
Cette partie sera faire dans RStudio